<a href="https://colab.research.google.com/github/seraogianluca/mircv-exercises/blob/main/Exercise5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#To support GPU
!pip install opencv-python==4.4.0.46

In [ ]:
import cv2
import os
import glob

import numpy as np

from PIL import Image
from IPython.display import clear_output, display
from imutils.video import VideoStream, FileVideoStream
from tqdm.notebook import tqdm


BASE_DIR = '/content/gdrive/My Drive/mircv2021/data/facerecognition'

#Detector parameters
DET_PROTO = BASE_DIR + '/caffe/face_detector/deploy.prototxt'
DET_MODEL = BASE_DIR + '/caffe/face_detector/res10_300x300_ssd_iter_140000_fp16.caffemodel'
DET_LAYER = 'detection_out'
DET_SIZE = (300, 300)
DET_MEAN = (123, 177,104)
DET_THRESHOLD = 0.7

#VGG2 parameters
FEAT_PROTO = BASE_DIR + '/caffe/face_features/resnet50_ft.prototxt'
FEAT_MODEL = BASE_DIR + '/caffe/face_features/resnet50_ft.caffemodel'
FEAT_LAYER = "pool5/7x7_s1"
FEAT_SIZE = (224, 224)
FEAT_MEAN = (91.4953, 103.8827, 131.0912)
PRED_THRESHOLD = 0.7

SRC_FOLDER = BASE_DIR + '/classes'

#k-Nearest Neighbors
VIDEO_PATH = BASE_DIR +  "/videos/dimartedi.mp4"
FRAMES_TO_SKIP = 800

GREEN = (0, 255, 0)
RED = (0, 0, 255)

In [ ]:
def highlight(img, rect, color, text=None):
    cv2.rectangle(img, rect[0], rect[1], color, 3, 3, 0)
    point = (rect[0][0] - 20, rect[1][1] + 50)
    if text != None:
        cv2.putText(img, text, point, 4, 1.2, color)


def getImageROI(img, face):
    return img[ face[0][1]:face[1][1], face[0][0]:face[1][0]]


def display_img(img, is_bgr=True):
  if is_bgr:  # convert color from CV2 BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)    
  display(Image.fromarray(img))
  clear_output(wait=True)

In [ ]:
class DNNExtractor:    
    
    def __init__(self, net_proto_path, trained_model_path, size, mean_values=None):
        self.size = size
        self.mean_values = mean_values

        self.net = cv2.dnn.readNetFromCaffe(net_proto_path, trained_model_path)
        # to enable GPU (this won't work on Colab without recompiling opencv)
        self.net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        self.net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        
    
    def extract(self, img, layer, normalize=False):
        blob = cv2.dnn.blobFromImage(img, 1.0, self.size, self.mean_values, swapRB=False, crop=False)
        self.net.setInput(blob)
        prob = self.net.forward(layer).flatten()

        if normalize:
            prob /= np.linalg.norm(prob)

        return prob
    
# it creates an instance of the class DNNExtractor
det = DNNExtractor(DET_PROTO, DET_MODEL, DET_SIZE, DET_MEAN)
fe = DNNExtractor(FEAT_PROTO, FEAT_MODEL, FEAT_SIZE, FEAT_MEAN)

In [ ]:
def extract_features(img_folder):
    # find all jpgs inside a subfolder
    img_paths = glob.glob(SRC_FOLDER + "/*/*.jpg")
    img_paths = sorted(img_paths)

    # get the id from filename: 22301.lavia.jpg --> lavia
    ids = [os.path.basename(path).split('.')[1] for path in img_paths]

    # extract features
    descs = [fe.extract(cv2.imread(path), FEAT_LAYER, normalize=True) for path in tqdm(img_paths)]

    return np.array(descs), np.array(ids)

descs, ids = extract_features(SRC_FOLDER)

In [ ]:
#TODO: STEP 1 - Detection

#detection is a vector of values
#please note that the detection coordinates are normalized. Revert back to the original coordinate system before putting them on the faces
#[batchId, classId, confidence, left, top, right, bottom]
def detect(img):
  bbs = det.extract(img, DET_LAYER)
  bbs = bbs.reshape(-1, 7)  # one detection per row
  h,w, _ = img.shape
    
  faces = []

  for batch_id, class_id, confidence, left, top, right, bottom in bbs:
    #check if the confidence value <  DET_THRESHOLD
    if confidence > DET_THRESHOLD:
    #determine points p0 and p1, i.e. the top left and bottom right vertex of the BB, with respect to the original coordinates from the normalized dimensions: left, top, right, bottom 
      p0 = (int(left*w), int(top*h))
      p1 = (int(right*w), int(bottom*h))
      faces.append((p0,p1))
  return faces

In [ ]:
#TODO: STEP 2 - Bounding Boxes
img = cv2.imread(os.path.join(BASE_DIR, 'lenna.jpg'))
faces = detect(img)
#for each face use highlight function to draw the face BB
for face in faces:
  highlight(img, face, GREEN, None)

#call display_img to display the detected faces
display_img(img)

In [ ]:
#TODO: STEP 3 - 1NN Classifier

class OneNNClassifier:
    
  def __init__(self, ids, descs):
      self.ids = ids
      self.descs = descs

  def predict(self, queryF):
    #evaluate the dot products between descs and queryF
    dots = np.dot(self.descs, queryF.transpose())
    #To sort the results create a zip between dot products and ids
    res = zip(dots, self.ids)
    #then call the sorted function to sort them
    res = sorted(res, reverse=True)
    #return just the first result of the results list
    return res[0]

In [ ]:
classifier = OneNNClassifier(ids, descs)

In [ ]:
#TODO: STEP 4 - Video face detection & recognition

def play_video(video_path, classifier):
  frame_count = 0

  print("[INFO] starting " + video_path)
  #vs = VideoStream(src=0).start() #webcam
  vs = FileVideoStream(video_path).start()

  # loop over frames from the video file stream
  while vs.more():

      # grab the frame from the threaded video stream
      frame = vs.read()

      if frame_count % FRAMES_TO_SKIP == 0:
          bb_color = GREEN
          detected_faces = detect(frame)
          #foreach detected face
          for face in detected_faces:
            #get the image ROI
            query = getImageROI(frame, face)
            #extract face features
            query_f = fe.extract(query,FEAT_LAYER,True)
            #classify
            confidence, prediction = classifier.predict(query_f)
            #highlight the detected face
            if confidence > PRED_THRESHOLD:
              highlight(frame, face, bb_color, prediction)
            else:
              highlight(frame, face, RED, "Unkown")
          display_img(frame)
          
      frame_count += 1

In [ ]:
play_video(VIDEO_PATH, classifier)

In [ ]:
#TODO: STEP 5 -  Nearest centroid classifier

mean_descs = []
mean_ids = []

#evaluate the mean feature for each class
#set the class ids
mean_ids = list(set(ids))
zipped = list(zip(ids, descs))

for i in range(len(mean_ids)):
  id = mean_ids[i]
  temp = [value for key,value in zipped if key == id]
  mean_descs.append(np.mean(temp, axis=0))

#init OneNNClassifier to make use of mean features
nearest_centroid_classifier = OneNNClassifier(mean_ids, mean_descs)

In [ ]:
play_video(VIDEO_PATH, nearest_centroid_classifier)